# Izgradite model klasifikacije: Ukusne azijske i indijske kuhinje


## Klasifikatori kuhinja 2

U ovoj drugoj lekciji o klasifikaciji, istražit ćemo `više načina` za klasifikaciju kategorijskih podataka. Također ćemo naučiti o posljedicama odabira jednog klasifikatora u odnosu na drugi.

### [**Kviz prije predavanja**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Preduvjeti**

Pretpostavljamo da ste završili prethodne lekcije jer ćemo nastaviti s nekim konceptima koje smo ranije naučili.

Za ovu lekciju trebat će nam sljedeći paketi:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [zbirka R paketa](https://www.tidyverse.org/packages) dizajnirana da učini znanost o podacima bržom, lakšom i zabavnijom!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) okvir je [zbirka paketa](https://www.tidymodels.org/packages/) za modeliranje i strojno učenje.

-   `themis`: [themis paket](https://themis.tidymodels.org/) pruža dodatne korake za recepte za rad s neuravnoteženim podacima.

Možete ih instalirati pomoću:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativno, skripta ispod provjerava imate li potrebne pakete za dovršavanje ovog modula i instalira ih za vas ako nedostaju.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Karta klasifikacije**

U našoj [prethodnoj lekciji](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), pokušali smo odgovoriti na pitanje: kako odabrati između više modela? U velikoj mjeri, to ovisi o karakteristikama podataka i vrsti problema koji želimo riješiti (na primjer, klasifikacija ili regresija?).

Ranije smo naučili o raznim opcijama koje imate pri klasifikaciji podataka koristeći Microsoftov vodič. Pythonov okvir za strojno učenje, Scikit-learn, nudi sličan, ali detaljniji vodič koji može dodatno pomoći u sužavanju izbora vaših procjenitelja (drugi izraz za klasifikatore):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Savjet: [posjetite ovu kartu online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) i kliknite na stazu kako biste pročitali dokumentaciju.
>
> [Referentna stranica Tidymodels](https://www.tidymodels.org/find/parsnip/#models) također pruža izvrsnu dokumentaciju o različitim vrstama modela.

### **Plan** 🗺️

Ova karta je vrlo korisna kada imate jasno razumijevanje svojih podataka, jer možete 'prošetati' njenim stazama do odluke:

-   Imamo \>50 uzoraka

-   Želimo predvidjeti kategoriju

-   Imamo označene podatke

-   Imamo manje od 100K uzoraka

-   ✨ Možemo odabrati Linear SVC

-   Ako to ne uspije, budući da imamo numeričke podatke

    -   Možemo pokušati ✨ KNeighbors Classifier

        -   Ako to ne uspije, pokušajte ✨ SVC i ✨ Ensemble Classifiers

Ovo je vrlo koristan put za slijediti. Sada, krenimo odmah koristeći [tidymodels](https://www.tidymodels.org/) okvir za modeliranje: dosljednu i fleksibilnu kolekciju R paketa razvijenih za poticanje dobre statističke prakse 😊.

## 2. Podijelite podatke i riješite problem neuravnoteženog skupa podataka.

Iz naših prethodnih lekcija naučili smo da postoji skup zajedničkih sastojaka među našim kuhinjama. Također, postojala je prilično nejednaka distribucija u broju kuhinja.

S ovim ćemo se nositi tako što ćemo:

-   Izbaciti najčešće sastojke koji stvaraju zabunu između različitih kuhinja, koristeći `dplyr::select()`.

-   Koristiti `recipe` koji unaprijed obrađuje podatke kako bi ih pripremio za modeliranje primjenom algoritma `over-sampling`.

Već smo pogledali gore navedeno u prethodnoj lekciji, tako da bi ovo trebalo biti jednostavno 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Rješavanje neuravnoteženih podataka

Neuravnoteženi podaci često negativno utječu na performanse modela. Mnogi modeli najbolje funkcioniraju kada je broj opažanja jednak i, stoga, imaju poteškoća s neuravnoteženim podacima.

Postoje dva glavna načina za rješavanje neuravnoteženih skupova podataka:

-   dodavanje opažanja manjinskoj klasi: `Over-sampling`, npr. korištenjem SMOTE algoritma koji sintetički generira nove primjere manjinske klase koristeći najbliže susjede tih slučajeva.

-   uklanjanje opažanja iz većinske klase: `Under-sampling`

U našoj prethodnoj lekciji pokazali smo kako se nositi s neuravnoteženim skupovima podataka koristeći `recipe`. Recipe se može smatrati nacrtom koji opisuje koje korake treba primijeniti na skup podataka kako bi bio spreman za analizu podataka. U našem slučaju, želimo postići jednaku raspodjelu broja naših kuhinja za naš `training set`. Krenimo odmah.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Sada smo spremni za treniranje modela 👩‍💻👨‍💻!

## 3. Izvan multinomijalnih regresijskih modela

U prethodnoj lekciji smo proučavali multinomijalne regresijske modele. Pogledajmo neke fleksibilnije modele za klasifikaciju.

### Strojevi za potporne vektore

U kontekstu klasifikacije, `Strojevi za potporne vektore` su tehnika strojnog učenja koja pokušava pronaći *hiperplohu* koja "najbolje" razdvaja klase. Pogledajmo jednostavan primjer:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ ne razdvaja klase. H2~ razdvaja, ali samo s malim razmakom. H3~ razdvaja s maksimalnim razmakom.

#### Linearni Support Vector Classifier

Support-Vector clustering (SVC) je dio obitelji Support-Vector strojeva (SVM) u tehnikama strojnog učenja. U SVC-u, hiperplana se odabire tako da ispravno razdvoji `većinu` promatranja u skupu za treniranje, ali `može pogrešno klasificirati` neka promatranja. Dopuštanjem da neke točke budu na pogrešnoj strani, SVM postaje otporniji na iznimke, čime se poboljšava generalizacija na nove podatke. Parametar koji regulira ovo odstupanje naziva se `cost`, a njegova zadana vrijednost je 1 (vidi `help("svm_poly")`).

Kreirajmo linearni SVC postavljanjem `degree = 1` u polinomni SVM model.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Sada kada smo obuhvatili korake predobrade i specifikaciju modela u *workflow*, možemo krenuti s treniranjem linearne SVC i procjenom rezultata u isto vrijeme. Za metrike izvedbe, kreirajmo skup metrika koji će procjenjivati: `točnost`, `osjetljivost`, `Pozitivnu prediktivnu vrijednost` i `F mjeru`.

> `augment()` će dodati stupac(e) za predikcije u dane podatke.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Strojevi potpornih vektora

Stroj potpornih vektora (SVM) je proširenje klasifikatora potpornih vektora kako bi se omogućila nelinearna granica između klasa. U suštini, SVM-ovi koriste *kernel trik* za proširenje prostora značajki kako bi se prilagodili nelinearnim odnosima između klasa. Jedna popularna i izuzetno fleksibilna kernel funkcija koju SVM-ovi koriste je *Radijalna bazna funkcija.* Pogledajmo kako će se ponašati na našim podacima.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Mnogo bolje 🤩!

> ✅ Molimo pogledajte:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> za dodatno čitanje.

### Klasifikatori najbližeg susjeda

*K*-najbliži susjed (KNN) je algoritam u kojem se svaka opažanja predviđa na temelju njezine *sličnosti* s drugim opažanjima.

Hajdemo ga primijeniti na naše podatke.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Čini se da ovaj model ne daje najbolje rezultate. Vjerojatno će promjena argumenata modela (pogledajte `help("nearest_neighbor")`) poboljšati performanse modela. Svakako isprobajte.

> ✅ Molimo pogledajte:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> kako biste saznali više o *K*-Nearest Neighbors klasifikatorima.

### Ansambl klasifikatori

Ansambl algoritmi rade tako da kombiniraju više osnovnih procjenitelja kako bi proizveli optimalan model, bilo:

`bagging`: primjenom *funkcije prosjeka* na zbirku osnovnih modela

`boosting`: izgradnjom niza modela koji se nadovezuju jedan na drugi kako bi poboljšali prediktivnu učinkovitost.

Počnimo s isprobavanjem modela Random Forest, koji gradi veliku zbirku stabala odluke, a zatim primjenjuje funkciju prosjeka za bolji ukupni model.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Dobar posao 👏!

Hajdemo također eksperimentirati s modelom Boosted Tree.

Boosted Tree definira ansambl metodu koja stvara niz sekvencijalnih stabala odluke gdje svako stablo ovisi o rezultatima prethodnih stabala u pokušaju da se postupno smanji pogreška. Fokusira se na težine pogrešno klasificiranih stavki i prilagođava model za sljedeći klasifikator kako bi ispravio.

Postoje različiti načini za prilagodbu ovog modela (vidi `help("boost_tree")`). U ovom primjeru, prilagodit ćemo Boosted Tree putem `xgboost` motora.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Molimo pogledajte:
>
> -   [Strojno učenje za društvene znanstvenike](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Praktično strojno učenje s R-om](https://bradleyboehmke.github.io/HOML/)
>
> -   [Uvod u statističko učenje s primjenama u R-u](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Istražuje model AdaBoost koji je dobra alternativa xgboostu.
>
> za više informacija o Ensemble klasifikatorima.

## 4. Dodatno - usporedba više modela

U ovom laboratoriju smo prilagodili priličan broj modela 🙌. Može postati zamorno ili zahtjevno kreirati mnogo radnih tijekova iz različitih skupova preprocesora i/ili specifikacija modela, a zatim pojedinačno izračunavati metrike izvedbe.

Pogledajmo možemo li to riješiti kreiranjem funkcije koja prilagođava popis radnih tijekova na skupu za treniranje, a zatim vraća metrike izvedbe na temelju skupa za testiranje. Koristit ćemo `map()` i `map_dfr()` iz paketa [purrr](https://purrr.tidyverse.org/) za primjenu funkcija na svaki element u popisu.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) funkcije omogućuju zamjenu mnogih for petlji kodom koji je sažetiji i lakši za čitanje. Najbolje mjesto za učenje o [`map()`](https://purrr.tidyverse.org/reference/map.html) funkcijama je [poglavlje o iteraciji](http://r4ds.had.co.nz/iteration.html) u R za znanost o podacima.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) paket omogućuje korisnicima kreiranje i jednostavno prilagođavanje velikog broja modela, ali je uglavnom dizajniran za rad s tehnikama uzorkovanja poput `križne validacije`, pristupa koji tek trebamo obraditi.

## **🚀Izazov**

Svaka od ovih tehnika ima veliki broj parametara koje možete prilagoditi, na primjer `cost` u SVM-ovima, `neighbors` u KNN-u, `mtry` (nasumično odabrani prediktori) u Random Forestu.

Istražite zadane parametre za svaki od njih i razmislite što bi prilagodba tih parametara značila za kvalitetu modela.

Da biste saznali više o određenom modelu i njegovim parametrima, koristite: `help("model")`, npr. `help("rand_forest")`

> U praksi obično *procjenjujemo* *najbolje vrijednosti* za ove parametre treniranjem mnogih modela na `simuliranom skupu podataka` i mjerenjem koliko dobro ti modeli rade. Ovaj proces se naziva **podešavanje**.

### [**Kviz nakon predavanja**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Pregled i samostalno učenje**

U ovim lekcijama ima puno stručnih izraza, pa odvojite trenutak da pregledate [ovaj popis](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) korisne terminologije!

#### ZAHVALA:

[`Allison Horst`](https://twitter.com/allison_horst/) za stvaranje nevjerojatnih ilustracija koje čine R pristupačnijim i zanimljivijim. Više ilustracija možete pronaći u njezinoj [galeriji](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) i [Jen Looper](https://www.twitter.com/jenlooper) za stvaranje originalne Python verzije ovog modula ♥️

Sretno u učenju,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Ilustracija @allison_horst</figcaption>



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije proizašle iz korištenja ovog prijevoda.
